In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shreenathbharadwaj","key":"14a5bd70d60f486fcbbee55000c32039"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                    title                                                size  lastUpdated          downloadCount  
-----------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
gpreda/covid-world-vaccination-progress                COVID-19 World Vaccination Progress                  41KB  2021-01-31 09:09:49           4679  
ayushggarg/all-trumps-twitter-insults-20152021         All Trump's Twitter insults (2015-2021)             581KB  2021-01-20 16:51:05            926  
sevgisarac/temperature-change                          Temperature change                                  778KB  2020-12-24 20:06:36           1002  
jorgesandoval/wind-power-generation                    Wind Power Generation Data                          245KB  2021-01-07 18:45:08            408  
cdminix/us-drought-meteorological-data                 US Drought & Meteorological Data       

In [8]:
! kaggle competitions download -c tabular-playground-series-jan-2021

 75% 18.0M/23.9M [00:00<00:00, 34.2MB/s]
100% 23.9M/23.9M [00:00<00:00, 48.7MB/s]
 65% 25.0M/38.4M [00:00<00:00, 42.0MB/s]
100% 38.4M/38.4M [00:00<00:00, 43.9MB/s]
  0% 0.00/469k [00:00<?, ?B/s]
100% 469k/469k [00:00<00:00, 155MB/s]


In [9]:
! mkdir data

In [10]:
! unzip train.csv.zip -d data
! unzip test.csv.zip -d data
! unzip sample_submission.csv.zip -d data

Archive:  train.csv.zip
  inflating: data/train.csv          
Archive:  test.csv.zip
  inflating: data/test.csv           
Archive:  sample_submission.csv.zip
  inflating: data/sample_submission.csv  


In [11]:
! pip install optuna

     |████████████████████████████████| 286kB 16.6MB/s 
     |████████████████████████████████| 1.1MB 56.0MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 481kB 55.2MB/s 
     |████████████████████████████████| 112kB 57.1MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 133kB 62.8MB/s 
  Created wheel for alembic: filename=alembic-1.5.3-py2.py3-none-any.whl size=155547 sha256=fc705f4c007693e708a6d23bcce81a5777a2c52e0607053d8ecc23cfa966a606
  Stored in directory: /root/.cache/pip/wheels/7f/ed/1b/a66a0cbca75fd3e374bd5cc60c443e5675f10ef1f1f78ec31d
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=b6c52e9bbd822c8617f5fe03b3caa035ecb04840943b3082a6961198f0f1d6ea
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 s

In [23]:
import numpy as np
import pandas as pd

import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import optuna
import xgboost as xgb

from sklearn.metrics import r2_score

In [13]:
train_df = pd.read_csv('data/train.csv', header = 0)
test_df = pd.read_csv('data/test.csv', header = 0)
train_df.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [14]:
train_df.shape

(300000, 16)

In [15]:
train_df.columns

Index(['id', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14',
       'target'],
      dtype='object')

In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cont1   300000 non-null  float64
 2   cont2   300000 non-null  float64
 3   cont3   300000 non-null  float64
 4   cont4   300000 non-null  float64
 5   cont5   300000 non-null  float64
 6   cont6   300000 non-null  float64
 7   cont7   300000 non-null  float64
 8   cont8   300000 non-null  float64
 9   cont9   300000 non-null  float64
 10  cont10  300000 non-null  float64
 11  cont11  300000 non-null  float64
 12  cont12  300000 non-null  float64
 13  cont13  300000 non-null  float64
 14  cont14  300000 non-null  float64
 15  target  300000 non-null  float64
dtypes: float64(15), int64(1)
memory usage: 36.6 MB


In [17]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,300000.0,249825.145857,144476.732562,1.000000,124656.500000,249738.500000,374836.250000,499999.000000
cont1,300000.0,0.506873,0.203976,-0.082263,0.343078,0.484005,0.643789,1.016227
cont2,300000.0,0.497898,0.228159,-0.031397,0.319170,0.553209,0.731263,0.859697
cont3,300000.0,0.521557,0.200770,0.020967,0.344096,0.551471,0.648315,1.006955
cont4,300000.0,0.515683,0.233035,0.152761,0.294935,0.482880,0.748705,1.010402
cont5,300000.0,0.502022,0.220701,0.276377,0.284108,0.451733,0.670660,1.034261
cont6,300000.0,0.526515,0.217909,0.066166,0.356163,0.470988,0.694043,1.043858
cont7,300000.0,0.487890,0.181096,-0.097666,0.346600,0.466825,0.581292,1.066167
cont8,300000.0,0.525163,0.216221,0.217260,0.341486,0.483460,0.685250,1.024427
cont9,300000.0,0.459857,0.196685,-0.240604,0.330832,0.416843,0.575041,1.004114


In [18]:
feature_cols = train_df.drop(['id', 'target'], axis=1).columns

x = train_df[feature_cols]
y = train_df['target']

print(x.shape, y.shape)

(300000, 14) (300000,)


In [19]:
param_grid = {
    'n_estimators': [5, 10, 15, 20],
    'max_depth': [2, 5, 7, 9]
}

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Instantiate model with 100 decision trees
clf = xgb.XGBRegressor(random_state = 42)

clf.fit(x_train, y_train)

[15:50:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [20]:
# Use the forest's predict method on the test data
predictions = clf.predict(x_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.6 degrees.


In [21]:
def objective(trial,data=x,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    
    # To select which parameters to optimize, please look at the XGBoost documentation:
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    param = {
        'tree_method':'gpu_hist',  # Use GPU acceleration
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [24]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2021-01-31 15:52:37,234] A new study created in memory with name: no-name-2b077543-1b54-42c5-b275-5806e5f57e2c


[15:52:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-01-31 15:52:41,208] Trial 0 finished with value: 0.8582729655857633 and parameters: {'lambda': 3.1405930056653766, 'alpha': 1.1260925251638223, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.018, 'n_estimators': 150, 'max_depth': 13, 'min_child_weight': 238}. Best is trial 0 with value: 0.8582729655857633.


[15:52:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-01-31 15:52:43,209] Trial 1 finished with value: 0.9805189257722055 and parameters: {'lambda': 0.2070427922644511, 'alpha': 2.8713128666404284, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.008, 'n_estimators': 300, 'max_depth': 4, 'min_child_weight': 224}. Best is trial 0 with value: 0.8582729655857633.


...

[16:04:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-01-31 16:05:06,523] Trial 23 finished with value: 0.6939268198203813 and parameters: {'lambda': 0.2510369635972826, 'alpha': 2.9259005166978107, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.01, 'n_estimators': 3000, 'max_depth': 15, 'min_child_weight': 171}. Best is trial 22 with value: 0.6938753178191799.


[16:05:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-01-31 16:05:47,324] Trial 24 finished with value: 0.694067918999114 and parameters: {'lambda': 0.17421852389113557, 'alpha': 2.2602325045063836, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.01, 'n_estimators': 3000, 'max_depth': 15, 'min_child_weight': 167}. Best is trial 22 with value: 0.6938753178191799.


Number of finished trials: 25
Best trial: {'lambda': 0.36311467811954184, 'alpha': 0.7114423595870062, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.01, 'n_estimators': 3000, 'max_depth': 15, 'min_child_weight': 152}


In [25]:
study.trials_dataframe().head()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_n_estimators,params_subsample,state
0,0,0.858273,2021-01-31 15:52:37.236719,2021-01-31 15:52:41.207205,0 days 00:00:03.970486,1.126093,0.5,3.140593,0.018,13,238,150,0.8,COMPLETE
1,1,0.980519,2021-01-31 15:52:41.209670,2021-01-31 15:52:43.208596,0 days 00:00:01.998926,2.871313,1.0,0.207043,0.008,4,224,300,1.0,COMPLETE
2,2,0.865524,2021-01-31 15:52:43.209910,2021-01-31 15:53:06.509397,0 days 00:00:23.299487,0.004314,0.8,0.007819,0.018,15,1,150,0.7,COMPLETE
3,3,0.698916,2021-01-31 15:53:06.510867,2021-01-31 15:53:11.446239,0 days 00:00:04.935372,0.004065,0.9,0.134375,0.020,9,98,300,0.8,COMPLETE
4,4,0.702087,2021-01-31 15:53:11.447528,2021-01-31 15:53:14.579013,0 days 00:00:03.131485,7.025211,0.9,0.376888,0.020,7,126,300,1.0,COMPLETE


In [26]:
# plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [27]:
# plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [28]:
# plot feature importance for algorithm parameters
visualization.plot_param_importances(study)

NameError: ignored

In [29]:
# finally plot best parameters
study.best_params

{'alpha': 0.7114423595870062,
 'colsample_bytree': 0.8,
 'lambda': 0.36311467811954184,
 'learning_rate': 0.01,
 'max_depth': 15,
 'min_child_weight': 152,
 'n_estimators': 3000,
 'subsample': 0.6}

In [30]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [32]:
best_params = study.best_params
best_params['tree_method'] = 'gpu_hist'
best_params['random_state'] = 42

clf = xgb.XGBRegressor(**(best_params))

clf.fit(x, y)

[16:06:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.7114423595870062, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             gamma=0, importance_type='gain', lambda=0.36311467811954184,
             learning_rate=0.01, max_delta_step=0, max_depth=15,
             min_child_weight=152, missing=None, n_estimators=3000, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=0.6, tree_method='gpu_hist', verbosity=1)

In [33]:
preds = pd.Series(clf.predict(test_df.drop('id', axis=1)), name='target')
preds = pd.concat([test_df['id'], preds], axis=1)

In [34]:
preds.head()

,id,target
0,0,8.011963
1,2,8.013160
2,6,7.908397
3,7,8.336289
4,10,8.023216


In [35]:
preds.to_csv("submission.csv", index=False)